First, download datasets (It will take a few minutes. You can comment out some of the files if you want a smaller dataset):

In [14]:
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S02E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S03E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S04E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S05E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S06E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S07E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S08E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S09E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S10E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S11E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S12E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S13E.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14T.mat
!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S14E.mat

--2018-03-10 00:30:31--  http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat
Resolving bnci-horizon-2020.eu (bnci-horizon-2020.eu)... 91.227.204.35
Connecting to bnci-horizon-2020.eu (bnci-horizon-2020.eu)|91.227.204.35|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat [following]
--2018-03-10 00:30:32--  https://lampx.tugraz.at/~bci/database/002-2014/S01T.mat
Resolving lampx.tugraz.at (lampx.tugraz.at)... 129.27.124.207
Connecting to lampx.tugraz.at (lampx.tugraz.at)|129.27.124.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39794870 (38M)
Saving to: ‘S01T.mat’

S01T.mat            100%[===================>]  37.95M  10.3MB/s    in 4.8s    



Move files to a separate folder:

In [0]:
!mkdir BBCIData
!mv *.mat BBCIData

Install dependencies (braindecode & pytorch):

In [0]:
!pip install braindecode -q

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

Now, let's load data (If you didn't download the all the files, comment out respective files).

We read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
    - the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref: Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf

In [45]:
import scipy.io as sio
import numpy as np
from os import listdir
from os.path import isfile, join

# prepare data containers
y = []
X = []

folder = "BBCIData"

for f in listdir(folder):
    # read file
    d1T = sio.loadmat(folder +  + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = 5*samplingRate


    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

# arrange data into numpy arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X = np.array(X).astype(np.float32)
y = (np.array(y).flatten()-1).astype(np.int64)
print(X.shape)
print(y.shape)

(2240, 15, 2560)
(2240,)


Load the models:

In [0]:
from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds    
from torch import optim
import torch

idx = np.random.permutation(X.shape[0])

X = X[idx,:,:]
y = y[idx]

nb_train_trials = int(np.floor(5/8*X.shape[0]))


train_set = SignalAndTarget(X[:nb_train_trials], y=y[:nb_train_trials])
test_set = SignalAndTarget(X[nb_train_trials:], y=y[nb_train_trials:])

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = torch.cuda.is_available()
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto').create_network()
if cuda:
    model.cuda()

Load optimizer. You can find hyperparameters in the link below.  
http://pytorch.org/docs/master/optim.html

In [0]:
optimizer = optim.Adam(model.parameters())

Finally start training:

In [42]:
from braindecode.torch_ext.util import np_to_var, var_to_np
from braindecode.datautil.iterators import get_balanced_batches
import torch.nn.functional as F
from numpy.random import RandomState
rng = RandomState(None)
#rng = RandomState((2017,6,30))
for i_epoch in range(50):
    i_trials_in_batch = get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                            batch_size=32)
    # Set model to training mode
    model.train()
    for i_trials in i_trials_in_batch:
        # Have to add empty fourth dimension to X
        batch_X = train_set.X[i_trials][:,:,:,None]
        batch_y = train_set.y[i_trials]
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        # Compute outputs of the network
        outputs = model(net_in)
        # Compute the loss
        loss = F.nll_loss(outputs, net_target)
        # Do the backpropagation
        loss.backward()
        # Update parameters with the optimizer
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set), ('Test', test_set)):
        i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, batch_size=32, shuffle=False)
        outputs = []
        net_targets = []
        for i_trials in i_trials_in_batch:
            batch_X = train_set.X[i_trials][:,:,:,None]
            batch_y = train_set.y[i_trials]
            
            net_in = np_to_var(batch_X)
            if cuda:
                net_in = net_in.cuda()
            net_target = np_to_var(batch_y)
            if cuda:
                net_target = net_target.cuda()
            net_target = var_to_np(net_target)
            output = var_to_np(model(net_in))
            outputs.append(output)
            net_targets.append(net_target)
        net_targets = np_to_var(np.concatenate(net_targets))
        outputs = np_to_var(np.concatenate(outputs))
        loss = F.nll_loss(outputs, net_targets)
        print("{:6s} Loss: {:.5f}".format(
            setname, float(var_to_np(loss))))
        predicted_labels = np.argmax(var_to_np(outputs), axis=1)
        accuracy = np.mean(dataset.y  == predicted_labels)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0
Train  Loss: 0.72573
Train  Accuracy: 55.4%
Test   Loss: 0.71195
Test   Accuracy: 49.6%
Epoch 1
Train  Loss: 0.65435
Train  Accuracy: 62.2%
Test   Loss: 0.64485
Test   Accuracy: 48.8%
Epoch 2
Train  Loss: 0.66047
Train  Accuracy: 61.1%
Test   Loss: 0.66806
Test   Accuracy: 49.0%
Epoch 3
Train  Loss: 0.65374
Train  Accuracy: 61.1%
Test   Loss: 0.66172
Test   Accuracy: 49.3%
Epoch 4
Train  Loss: 0.76140
Train  Accuracy: 60.6%
Test   Loss: 0.77328
Test   Accuracy: 51.0%
Epoch 5
Train  Loss: 0.69497
Train  Accuracy: 62.1%
Test   Loss: 0.69986
Test   Accuracy: 49.8%
Epoch 6
Train  Loss: 0.61175
Train  Accuracy: 65.6%
Test   Loss: 0.62332
Test   Accuracy: 49.3%
Epoch 7
Train  Loss: 0.58174
Train  Accuracy: 68.9%
Test   Loss: 0.57760
Test   Accuracy: 49.3%
Epoch 8
Train  Loss: 0.57410
Train  Accuracy: 68.3%
Test   Loss: 0.57376
Test   Accuracy: 49.4%
Epoch 9
Train  Loss: 0.60040
Train  Accuracy: 66.1%
Test   Loss: 0.57964
Test   Accuracy: 47.3%
Epoch 10
Train  Loss: 0.63183
Train  Acc

Epoch 32
Train  Loss: 0.58961
Train  Accuracy: 68.9%
Test   Loss: 0.59968
Test   Accuracy: 50.2%
Epoch 33
Train  Loss: 0.54916
Train  Accuracy: 70.6%
Test   Loss: 0.56440
Test   Accuracy: 50.8%
Epoch 34
Train  Loss: 0.48990
Train  Accuracy: 75.1%
Test   Loss: 0.50170
Test   Accuracy: 49.8%
Epoch 35
Train  Loss: 0.46224
Train  Accuracy: 77.6%
Test   Loss: 0.47194
Test   Accuracy: 49.6%
Epoch 36
Train  Loss: 0.46028
Train  Accuracy: 78.6%
Test   Loss: 0.45661
Test   Accuracy: 48.7%
Epoch 37
Train  Loss: 0.45067
Train  Accuracy: 80.1%
Test   Loss: 0.44671
Test   Accuracy: 47.1%
Epoch 38
Train  Loss: 0.46178
Train  Accuracy: 77.4%
Test   Loss: 0.47533
Test   Accuracy: 49.0%
Epoch 39
Train  Loss: 0.51030
Train  Accuracy: 75.1%
Test   Loss: 0.52092
Test   Accuracy: 50.0%
Epoch 40
Train  Loss: 0.45420
Train  Accuracy: 79.6%
Test   Loss: 0.45321
Test   Accuracy: 49.3%
Epoch 41
Train  Loss: 0.48343
Train  Accuracy: 76.6%
Test   Loss: 0.48765
Test   Accuracy: 50.6%
Epoch 42
Train  Loss: 0.47272
